In [1]:
import torch
import torch.nn as nn
import numpy as np
from model import CAEP
from utils import Kodak
from torchvision.utils import make_grid, save_image

In [111]:
print('Number of GPUs available: ' + str(torch.cuda.device_count()))
model = nn.DataParallel(CAEP(15).cuda())
pretrained_state_dict = torch.load(f"./chkpt/bpp_10_before_pruning/model.state")
current_state_dict = model.state_dict()
current_state_dict.update(pretrained_state_dict)
model.load_state_dict(current_state_dict)
model.eval()
print('Done Setup Model.')

Number of GPUs available: 1

Done Setup Model.

In [112]:
dataset = Kodak('./codevis/')

In [113]:
img = dataset[0][0]
patches = dataset[0][1]

In [114]:
patches.shape

torch.Size([6, 4, 3, 128, 128])

In [115]:
stacki = []
for i in range(6):
    stackj = []
    for j in range(4):
        x = torch.Tensor(patches[i, j, :, :, :].unsqueeze(0)).cuda()
        y, c = model(x)
        stackj.append(c.squeeze(0).cpu().data)
    stacki.append(torch.cat(stackj, dim=1))
out = torch.cat(stacki, dim=2)
out = out.unsqueeze(1)
grid = make_grid(out, nrow=6, normalize=True)

In [23]:
save_image(grid, './codevis/kodim23_code_before_pruning.png')

In [14]:
save_image(torch.Tensor([[0, -1], [1, 0]]), './codevis/test.png')  # confirm that non-positives are black

In [79]:
# ratio of zero elements before pruning
r_before_p = torch.sum(out == 0).numpy() / torch.numel(out)
print('Ratio of zero elements before pruning : %.2f%%' % (r_before_p * 100))

Ratio of zero elements before pruning : 9.60%

In [75]:
# ratio of zero elements after pruning 
# Remind to change model checkpoint!!
r_after_p = torch.sum(out == 0).numpy() / torch.numel(out)
print('Ratio of zero elements after pruning : %.2f%%' % (r_after_p * 100))

Ratio of zero elements after pruning : 21.51%

In [80]:
# the increased ratio of zeros
relative_change = (r_after_p - r_before_p) / r_before_p
print('The increased ratio of zeros : %.2f %%' % (relative_change * 100))

The increased ratio of zeros : 124.07 %

In [81]:
# the decreased ratio of non-zeros
relative_change_non_zero = ((1 - r_after_p) - (1 - r_before_p)) / (1 - r_before_p)
print('The decreased ratio of non-zeros : %.2f %%' % (relative_change_non_zero * 100))

The decreased ratio of non-zeros : -13.18 %

In [116]:
# mask those zero elements
def norm_ip(img, min, max):
    img.clamp_(min=min, max=max)
    img.add_(-min).div_(max - min + 1e-5)


def norm_range(t):
    t = t.clone()
    norm_ip(t, float(t.min()), float(t.max()))
    return t
        

norm = norm_range(out)
masked = norm.masked_fill_(out==0, 0)

In [117]:
save_image(make_grid(masked, nrow=6), './codevis/kodim23_masked_code_before_pruning.png')